In [2]:
#GPU count and name
!nvidia-smi -L

GPU 0: GeForce RTX 2080 Ti (UUID: GPU-616398ce-ae25-c52b-7c47-f658d30d9d86)


In [ ]:
!pip install tensorflow-gpu==2.2.0

In [ ]:
!pip install Keras==2.3.1

In [1]:
!python -c 'import keras; print(keras.__version__)'

Using TensorFlow backend.
2.3.1


## Set paths

In [30]:
import os
DATASET_BASE_DIR = '../datasets/data'
DATASET_VERSION = 'v4'
DATASET_VERSION_DIR = os.path.join(DATASET_BASE_DIR, DATASET_VERSION)
CLASSES_FILE = os.path.join(DATASET_VERSION_DIR, 'classes.csv')
TRAIN_ANNOTATIONS = os.path.join(DATASET_VERSION_DIR, 'train.csv')
VAL_ANNOTATIONS = os.path.join(DATASET_VERSION_DIR, 'val.csv')

TRAINING_BASE_DIR = os.path.join('../trainings', DATASET_VERSION)

## Check directories and files

In [3]:
base_dir = os.getcwd()
base_dir

'/home/aikauel/enap/aerialnet_project/notebooks'

In [4]:
%ls

 alertas_07-08-2020.json         Enap_Dataset_Formatting_v3.ipynb
 BlobsExtraction.ipynb           RetinaNet_v3.ipynb
'Dibujar predicciones.ipynb'     RetinaNet_v4.ipynb
 Enap_Dataset_Formatting.ipynb   visualization.ipynb


In [31]:
!wc -l {TRAIN_ANNOTATIONS}

20574 ../datasets/data/v4/train.csv


In [29]:
!wc -l {VAL_ANNOTATIONS}

1346 ../datasets/data/v4/val.csv


In [32]:
!wc -l {CLASSES_FILE}

9 ../datasets/data/v4/classes.csv


## Set pretrained model

In [39]:
# utilize best weights from 4 classes model as baseline
PRETRAINED_MODEL = '/home/aikauel/enap/aerialnet_project/trainings/v3/snapshots_iter2/resnet50_csv_108.h5'
BATCH_SIZE = 8

# Compute best anchors for dataset annotations

In [14]:
!anchor-optimization {TRAIN_ANNOTATIONS} --no-resize --ratios=9 --threads 100

Using TensorFlow backend.
Optimization ended successfully!

Final best anchor configuration
State: 0.12508
Ratios: [0.25, 0.403, 0.567, 0.763, 1.0, 1.311, 1.762, 2.481, 4.0]
Scales: [0.656, 0.824, 1.037]
Number of labels that don't have any matching anchor: 86


Save anchor optimization to config.ini file

# Set number of steps per epoch

In [40]:
import pandas as pd
import math
# compute number of annotations to compute number of steps per epoch
df = pd.read_csv(TRAIN_ANNOTATIONS, header=None, names=['img_path', 'x1', 'y1', 'x2', 'y2', 'class'])

countAnn = len(df)-1
countImg = df['img_path'].nunique()
no_steps = math.ceil(countImg/BATCH_SIZE)

print("Count of images: {}".format(countImg))
print("Count of annotations: {}".format(countAnn))
print("Number of steps per epoch: {}".format(no_steps))

Count of images: 4779
Count of annotations: 20574
Number of steps per epoch: 598


# Train model

In [24]:
CONFIG_FILE = os.path.join(TRAINING_BASE_DIR, 'config.ini')
TENSORBOARD_LOGS_DIR = os.path.join(TRAINING_BASE_DIR, 'logs')
SNAPSHOTS_DIR = os.path.join(TRAINING_BASE_DIR, 'snapshots')

In [25]:
TENSORBOARD_LOGS_DIR

'../trainings/v4/logs'

### FCNs training (regression, classification) with freezed backbone  

First, train the model freezing the backbone. The backbone weights will be the weights from the previous model train before.
Set gamma with high value (>2.0) to weight the loss of well-classified classes down, forcing the model to learn on harder (in this case: less examples) classes.

In [34]:
!python ../../keras-retinanet/keras_retinanet/bin/train.py \
--config {CONFIG_FILE} \
--freeze-backbone \
--random-transform \
--weights {PRETRAINED_MODEL} \
--weighted-average \
--batch-size {BATCH_SIZE} \
--steps {no_steps} \
--epochs 50 \
--no-resize \
--image-min-side 1000 \
--image-max-side 1000 \
--compute-val-loss \
--multiprocessing \
--workers 2 \
--tensorboard-dir {TENSORBOARD_LOGS_DIR} \
--snapshot-path {SNAPSHOTS_DIR} \
--validation-freq 5 \
--gamma 3.0 \
--class-names '/home/aikauel/enap/aerialnet_project/datasets/data/v4/CLASSES' \
csv {TRAIN_ANNOTATIONS} {CLASSES_FILE} \
--val-annotations {VAL_ANNOTATIONS}

Using TensorFlow backend.
Creating model, this may take a second...
2020-09-14 13:08:22.753420: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-09-14 13:08:22.783440: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-09-14 13:08:22.783812: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.755GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2020-09-14 13:08:22.783965: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-09-14 13:08:22.784851: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.s

tracking <tf.Variable 'Variable:0' shape=(15, 4) dtype=float32, numpy=
array([[-33.472   ,  -8.368   ,  33.472   ,   8.368   ],
       [-52.992   , -13.248   ,  52.992   ,  13.248   ],
       [-84.864   , -21.216   ,  84.864   ,  21.216   ],
       [-21.733232, -12.887807,  21.733232,  12.887807],
       [-34.40749 , -20.40364 ,  34.40749 ,  20.40364 ],
       [-55.101845, -32.675396,  55.101845,  32.675396],
       [-16.736   , -16.736   ,  16.736   ,  16.736   ],
       [-26.496   , -26.496   ,  26.496   ,  26.496   ],
       [-42.432   , -42.432   ,  42.432   ,  42.432   ],
       [-12.889109, -21.731037,  12.889109,  21.731037],
       [-20.405703, -34.404015,  20.405703,  34.404015],
       [-32.678696, -55.09628 ,  32.678696,  55.09628 ],
       [ -8.368   , -33.472   ,   8.368   ,  33.472   ],
       [-13.248   , -52.992   ,  13.248   ,  52.992   ],
       [-21.216   , -84.864   ,  21.216   ,  84.864   ]], dtype=float32)> anchors
tracking <tf.Variable 'Variable:0' shape=(15, 4) 

Epoch 1/50
2020-09-14 13:08:32.208936: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2020-09-14 13:08:33.551642: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
  1/598 [..............................] - ETA: 1:38:45 - loss: 2.8596 - regression_loss: 1.7327 - classification_loss: 1.12692020-09-14 13:08:37.081814: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session started.
2020-09-14 13:08:37.081849: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1408] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI could not be loaded or symbol could not be found.
2020-09-14 13:08:37.081858: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1447] function cupti_interface_->ActivityRegisterCallbacks( AllocCuptiActivityBuffer, FreeCuptiActivityBuffer)failed with error C

Epoch 4/50
598/598 [==============================] - 424s 709ms/step - loss: 1.3506 - regression_loss: 1.1548 - classification_loss: 0.1958
Running network: 100% (612 of 612) |#####| Elapsed Time: 0:00:53 Time:  0:00:53
Parsing annotations: 100% (612 of 612) |#| Elapsed Time: 0:00:00 Time:  0:00:00
218 instances of class Vehículo with average precision: 0.9292 TP: 213 FP: 1731 precision: 0.1096 recall: 0.9771 f1_score: 0.1970 f2_score: 0.3782
16 instances of class Camión with average precision: 0.4260 TP: 14 FP: 876 precision: 0.0157 recall: 0.8750 f1_score: 0.0309 f2_score: 0.0734
7 instances of class Tractor with average precision: 0.2110 TP: 6 FP: 381 precision: 0.0155 recall: 0.8571 f1_score: 0.0305 f2_score: 0.0723
8 instances of class Maquinaria with average precision: 0.5809 TP: 8 FP: 368 precision: 0.0213 recall: 1.0000 f1_score: 0.0417 f2_score: 0.0980
68 instances of class Animal with average precision: 0.6764 TP: 60 FP: 5834 precision: 0.0102 recall: 0.8824 f1_score: 0.0201

Epoch 9/50
598/598 [==============================] - 450s 753ms/step - loss: 1.3075 - regression_loss: 1.1280 - classification_loss: 0.1795 - val_loss: 1.8670 - val_regression_loss: 1.3976 - val_classification_loss: 1.0214
Running network: 100% (612 of 612) |#####| Elapsed Time: 0:00:53 Time:  0:00:53
Parsing annotations: 100% (612 of 612) |#| Elapsed Time: 0:00:00 Time:  0:00:00
218 instances of class Vehículo with average precision: 0.9338 TP: 213 FP: 1608 precision: 0.1170 recall: 0.9771 f1_score: 0.2089 f2_score: 0.3955
16 instances of class Camión with average precision: 0.4392 TP: 14 FP: 662 precision: 0.0207 recall: 0.8750 f1_score: 0.0405 f2_score: 0.0946
7 instances of class Tractor with average precision: 0.2336 TP: 7 FP: 426 precision: 0.0162 recall: 1.0000 f1_score: 0.0318 f2_score: 0.0759
8 instances of class Maquinaria with average precision: 0.6443 TP: 8 FP: 593 precision: 0.0133 recall: 1.0000 f1_score: 0.0263 f2_score: 0.0632
68 instances of class Animal with average 

Epoch 14/50
598/598 [==============================] - 446s 746ms/step - loss: 1.2967 - regression_loss: 1.1248 - classification_loss: 0.1719 - val_loss: 1.8421 - val_regression_loss: 1.4482 - val_classification_loss: 0.8727
Running network: 100% (612 of 612) |#####| Elapsed Time: 0:00:53 Time:  0:00:53
Parsing annotations: 100% (612 of 612) |#| Elapsed Time: 0:00:00 Time:  0:00:00
218 instances of class Vehículo with average precision: 0.9232 TP: 211 FP: 1364 precision: 0.1340 recall: 0.9679 f1_score: 0.2354 f2_score: 0.4311
16 instances of class Camión with average precision: 0.4144 TP: 15 FP: 809 precision: 0.0182 recall: 0.9375 f1_score: 0.0357 f2_score: 0.0845
7 instances of class Tractor with average precision: 0.2582 TP: 7 FP: 453 precision: 0.0152 recall: 1.0000 f1_score: 0.0300 f2_score: 0.0717
8 instances of class Maquinaria with average precision: 0.6082 TP: 8 FP: 488 precision: 0.0161 recall: 1.0000 f1_score: 0.0317 f2_score: 0.0758
68 instances of class Animal with average

Epoch 19/50
598/598 [==============================] - 449s 751ms/step - loss: 1.2902 - regression_loss: 1.1188 - classification_loss: 0.1714 - val_loss: 1.8064 - val_regression_loss: 1.4076 - val_classification_loss: 0.9719
Running network: 100% (612 of 612) |#####| Elapsed Time: 0:00:54 Time:  0:00:54
Parsing annotations: 100% (612 of 612) |#| Elapsed Time: 0:00:00 Time:  0:00:00
218 instances of class Vehículo with average precision: 0.9278 TP: 212 FP: 1384 precision: 0.1328 recall: 0.9725 f1_score: 0.2337 f2_score: 0.4295
16 instances of class Camión with average precision: 0.4101 TP: 15 FP: 776 precision: 0.0190 recall: 0.9375 f1_score: 0.0372 f2_score: 0.0877
7 instances of class Tractor with average precision: 0.2553 TP: 7 FP: 440 precision: 0.0157 recall: 1.0000 f1_score: 0.0308 f2_score: 0.0737
8 instances of class Maquinaria with average precision: 0.6062 TP: 8 FP: 474 precision: 0.0166 recall: 1.0000 f1_score: 0.0327 f2_score: 0.0778
68 instances of class Animal with average

In [42]:
last_model = os.path.join(SNAPSHOTS_DIR, 'resnet50_csv_19.h5')

!python ../../keras-retinanet/keras_retinanet/bin/train.py \
--config {CONFIG_FILE} \
--freeze-backbone \
--random-transform \
--weights {last_model} \
--initial-epoch 19 \
--weighted-average \
--batch-size {BATCH_SIZE} \
--steps {no_steps} \
--epochs 50 \
--no-resize \
--image-min-side 1000 \
--image-max-side 1000 \
--compute-val-loss \
--multiprocessing \
--workers 2 \
--tensorboard-dir {TENSORBOARD_LOGS_DIR} \
--snapshot-path {SNAPSHOTS_DIR} \
--validation-freq 5 \
--gamma 3.0 \
--class-names '/home/aikauel/enap/aerialnet_project/datasets/data/v4/CLASSES' \
csv {TRAIN_ANNOTATIONS} {CLASSES_FILE} \
--val-annotations {VAL_ANNOTATIONS}

Using TensorFlow backend.
Creating model, this may take a second...
2020-09-14 17:01:05.602886: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-09-14 17:01:05.630908: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-09-14 17:01:05.631232: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.755GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2020-09-14 17:01:05.631402: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-09-14 17:01:05.632372: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.s

tracking <tf.Variable 'Variable:0' shape=(15, 4) dtype=float32, numpy=
array([[ -66.944   ,  -16.736   ,   66.944   ,   16.736   ],
       [-105.984   ,  -26.496   ,  105.984   ,   26.496   ],
       [-169.728   ,  -42.432   ,  169.728   ,   42.432   ],
       [ -43.466465,  -25.775614,   43.466465,   25.775614],
       [ -68.81498 ,  -40.80728 ,   68.81498 ,   40.80728 ],
       [-110.20369 ,  -65.35079 ,  110.20369 ,   65.35079 ],
       [ -33.472   ,  -33.472   ,   33.472   ,   33.472   ],
       [ -52.992   ,  -52.992   ,   52.992   ,   52.992   ],
       [ -84.864   ,  -84.864   ,   84.864   ,   84.864   ],
       [ -25.778217,  -43.462074,   25.778217,   43.462074],
       [ -40.811405,  -68.80803 ,   40.811405,   68.80803 ],
       [ -65.35739 , -110.19256 ,   65.35739 ,  110.19256 ],
       [ -16.736   ,  -66.944   ,   16.736   ,   66.944   ],
       [ -26.496   , -105.984   ,   26.496   ,  105.984   ],
       [ -42.432   , -169.728   ,   42.432   ,  169.728   ]],
      dtype=f

Epoch 20/50
2020-09-14 17:01:15.659885: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2020-09-14 17:01:16.936608: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
  1/598 [..............................] - ETA: 1:44:22 - loss: 1.8440 - regression_loss: 1.5650 - classification_loss: 0.27912020-09-14 17:01:20.334054: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session started.
2020-09-14 17:01:20.334088: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1408] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI could not be loaded or symbol could not be found.
2020-09-14 17:01:20.334096: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1447] function cupti_interface_->ActivityRegisterCallbacks( AllocCuptiActivityBuffer, FreeCuptiActivityBuffer)failed with error 

Epoch 23/50
598/598 [==============================] - 439s 735ms/step - loss: 1.2923 - regression_loss: 1.1213 - classification_loss: 0.1710 - val_loss: 1.7987 - val_regression_loss: 1.4193 - val_classification_loss: 0.9386
Running network: 100% (612 of 612) |#####| Elapsed Time: 0:00:53 Time:  0:00:53
Parsing annotations: 100% (612 of 612) |#| Elapsed Time: 0:00:00 Time:  0:00:00
218 instances of class Vehículo with average precision: 0.9300 TP: 188 FP: 20 precision: 0.9038 recall: 0.8624 f1_score: 0.8826 f2_score: 0.8704
16 instances of class Camión with average precision: 0.4446 TP: 4 FP: 1 precision: 0.8000 recall: 0.2500 f1_score: 0.3810 f2_score: 0.2899
7 instances of class Tractor with average precision: 0.1594 TP: 1 FP: 3 precision: 0.2500 recall: 0.1429 f1_score: 0.1818 f2_score: 0.1562
8 instances of class Maquinaria with average precision: 0.6220 TP: 3 FP: 0 precision: 1.0000 recall: 0.3750 f1_score: 0.5455 f2_score: 0.4286
68 instances of class Animal with average precisio

#### Full model training

With a decent mAP result in previous step, we now unfreeze the backbone weights to train the full model.

In [43]:
BATCH_SIZE = 4

# compute number of annotations to compute number of steps per epoch
df = pd.read_csv(TRAIN_ANNOTATIONS, header=None, names=['img_path', 'x1', 'y1', 'x2', 'y2', 'class'])

countAnn = len(df)-1
countImg = df['img_path'].nunique()
no_steps = math.ceil(countImg/BATCH_SIZE)

print("Count of images: {}".format(countImg))
print("Count of annotations: {}".format(countAnn))
print("Number of steps per epoch: {}".format(no_steps))

Count of images: 4779
Count of annotations: 20574
Number of steps per epoch: 1195


In [44]:
last_model = os.path.join(SNAPSHOTS_DIR, 'resnet50_csv_23.h5')

!python ../../keras-retinanet/keras_retinanet/bin/train.py \
--config {CONFIG_FILE} \
--random-transform \
--weights {last_model} \
--initial-epoch 23 \
--weighted-average \
--batch-size {BATCH_SIZE} \
--steps {no_steps} \
--epochs 100 \
--no-resize \
--image-min-side 1000 \
--image-max-side 1000 \
--compute-val-loss \
--multiprocessing \
--workers 2 \
--tensorboard-dir {TENSORBOARD_LOGS_DIR} \
--snapshot-path {SNAPSHOTS_DIR} \
--validation-freq 5 \
--lr 1e-5 \
--reduce-lr-patience 2 \
--reduce-lr-factor 0.1 \
--gamma 3.0 \
--class-names '/home/aikauel/enap/aerialnet_project/datasets/data/v4/CLASSES' \
csv {TRAIN_ANNOTATIONS} {CLASSES_FILE} \
--val-annotations {VAL_ANNOTATIONS}

Using TensorFlow backend.
Creating model, this may take a second...
2020-09-14 18:00:42.241151: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-09-14 18:00:42.271464: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-09-14 18:00:42.271840: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.755GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2020-09-14 18:00:42.272023: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-09-14 18:00:42.272942: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.s

tracking <tf.Variable 'Variable:0' shape=(15, 4) dtype=float32, numpy=
array([[ -66.944   ,  -16.736   ,   66.944   ,   16.736   ],
       [-105.984   ,  -26.496   ,  105.984   ,   26.496   ],
       [-169.728   ,  -42.432   ,  169.728   ,   42.432   ],
       [ -43.466465,  -25.775614,   43.466465,   25.775614],
       [ -68.81498 ,  -40.80728 ,   68.81498 ,   40.80728 ],
       [-110.20369 ,  -65.35079 ,  110.20369 ,   65.35079 ],
       [ -33.472   ,  -33.472   ,   33.472   ,   33.472   ],
       [ -52.992   ,  -52.992   ,   52.992   ,   52.992   ],
       [ -84.864   ,  -84.864   ,   84.864   ,   84.864   ],
       [ -25.778217,  -43.462074,   25.778217,   43.462074],
       [ -40.811405,  -68.80803 ,   40.811405,   68.80803 ],
       [ -65.35739 , -110.19256 ,   65.35739 ,  110.19256 ],
       [ -16.736   ,  -66.944   ,   16.736   ,   66.944   ],
       [ -26.496   , -105.984   ,   26.496   ,  105.984   ],
       [ -42.432   , -169.728   ,   42.432   ,  169.728   ]],
      dtype=f

Epoch 24/100
2020-09-14 18:00:57.929457: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2020-09-14 18:00:58.859528: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
   1/1195 [..............................] - ETA: 4:21:29 - loss: 1.1436 - regression_loss: 0.9903 - classification_loss: 0.15332020-09-14 18:01:02.265168: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session started.
2020-09-14 18:01:02.265203: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1408] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI could not be loaded or symbol could not be found.
2020-09-14 18:01:02.265211: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1447] function cupti_interface_->ActivityRegisterCallbacks( AllocCuptiActivityBuffer, FreeCuptiActivityBuffer)failed with err

Epoch 27/100
1195/1195 [==============================] - 615s 515ms/step - loss: 1.2779 - regression_loss: 1.1099 - classification_loss: 0.1680 - val_loss: 1.7563 - val_regression_loss: 1.0813 - val_classification_loss: 0.9877
Running network: 100% (612 of 612) |#####| Elapsed Time: 0:00:53 Time:  0:00:53
Parsing annotations: 100% (612 of 612) |#| Elapsed Time: 0:00:00 Time:  0:00:00
218 instances of class Vehículo with average precision: 0.9312 TP: 196 FP: 20 precision: 0.9074 recall: 0.8991 f1_score: 0.9032 f2_score: 0.9007
16 instances of class Camión with average precision: 0.4541 TP: 4 FP: 0 precision: 1.0000 recall: 0.2500 f1_score: 0.4000 f2_score: 0.2941
7 instances of class Tractor with average precision: 0.2258 TP: 1 FP: 1 precision: 0.5000 recall: 0.1429 f1_score: 0.2222 f2_score: 0.1667
8 instances of class Maquinaria with average precision: 0.6951 TP: 4 FP: 1 precision: 0.8000 recall: 0.5000 f1_score: 0.6154 f2_score: 0.5405
68 instances of class Animal with average preci

## ....

In [59]:
BATCH_SIZE = 8

TRAIN_ANNOTATIONS = os.path.join(DATASET_VERSION_DIR, 'train.csv')
VAL_ANNOTATIONS = os.path.join(DATASET_VERSION_DIR, 'val.csv')
# compute number of annotations to compute number of steps per epoch
df = pd.read_csv(TRAIN_ANNOTATIONS, header=None, names=['img_path', 'x1', 'y1', 'x2', 'y2', 'class'])

countAnn = len(df)-1
countImg = df['img_path'].nunique()
no_steps = math.ceil(countImg/BATCH_SIZE)

print("Count of images: {}".format(countImg))
print("Count of annotations: {}".format(countAnn))
print("Number of steps per epoch: {}".format(no_steps))

Count of images: 4779
Count of annotations: 20574
Number of steps per epoch: 598


In [60]:
epoch = 27
last_model = os.path.join(SNAPSHOTS_DIR, "resnet50_csv_{}.h5".format(epoch))

!python ../../keras-retinanet/keras_retinanet/bin/train.py \
--config {CONFIG_FILE} \
--freeze-backbone \
--random-transform \
--weights {last_model} \
--initial-epoch {epoch} \
--weighted-average \
--batch-size {BATCH_SIZE} \
--steps {no_steps} \
--epochs 50 \
--no-resize \
--image-min-side 1000 \
--image-max-side 1000 \
--compute-val-loss \
--multiprocessing \
--workers 2 \
--tensorboard-dir {TENSORBOARD_LOGS_DIR} \
--snapshot-path {SNAPSHOTS_DIR} \
--validation-freq 5 \
--gamma 3.0 \
--class-names '/home/aikauel/enap/aerialnet_project/datasets/data/v4/CLASSES' \
csv {TRAIN_ANNOTATIONS} {CLASSES_FILE} \
--val-annotations {VAL_ANNOTATIONS}

Using TensorFlow backend.
Creating model, this may take a second...
2020-09-15 00:40:08.016014: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-09-15 00:40:08.046731: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-09-15 00:40:08.047066: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.755GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2020-09-15 00:40:08.047226: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-09-15 00:40:08.048182: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.s

tracking <tf.Variable 'Variable:0' shape=(15, 4) dtype=float32, numpy=
array([[-133.888   ,  -33.472   ,  133.888   ,   33.472   ],
       [-211.968   ,  -52.992   ,  211.968   ,   52.992   ],
       [-339.456   ,  -84.864   ,  339.456   ,   84.864   ],
       [ -86.93293 ,  -51.551228,   86.93293 ,   51.551228],
       [-137.62996 ,  -81.61456 ,  137.62996 ,   81.61456 ],
       [-220.40738 , -130.70158 ,  220.40738 ,  130.70158 ],
       [ -66.944   ,  -66.944   ,   66.944   ,   66.944   ],
       [-105.984   , -105.984   ,  105.984   ,  105.984   ],
       [-169.728   , -169.728   ,  169.728   ,  169.728   ],
       [ -51.556435,  -86.92415 ,   51.556435,   86.92415 ],
       [ -81.62281 , -137.61606 ,   81.62281 ,  137.61606 ],
       [-130.71478 , -220.38512 ,  130.71478 ,  220.38512 ],
       [ -33.472   , -133.888   ,   33.472   ,  133.888   ],
       [ -52.992   , -211.968   ,   52.992   ,  211.968   ],
       [ -84.864   , -339.456   ,   84.864   ,  339.456   ]],
      dtype=f

Epoch 28/50
2020-09-15 00:40:17.467729: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2020-09-15 00:40:18.823126: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
  1/598 [..............................] - ETA: 1:38:22 - loss: 1.4590 - regression_loss: 1.1082 - classification_loss: 0.35082020-09-15 00:40:22.270563: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session started.
2020-09-15 00:40:22.270599: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1408] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI could not be loaded or symbol could not be found.
2020-09-15 00:40:22.270607: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1447] function cupti_interface_->ActivityRegisterCallbacks( AllocCuptiActivityBuffer, FreeCuptiActivityBuffer)failed with error 

Epoch 31/50
598/598 [==============================] - 439s 734ms/step - loss: 1.1897 - regression_loss: 1.0797 - classification_loss: 0.1100 - val_loss: 1.7283 - val_regression_loss: 1.4410 - val_classification_loss: 0.7084
Running network: 100% (612 of 612) |#####| Elapsed Time: 0:00:52 Time:  0:00:52
Parsing annotations: 100% (612 of 612) |#| Elapsed Time: 0:00:00 Time:  0:00:00
218 instances of class Vehículo with average precision: 0.9327 TP: 200 FP: 23 precision: 0.8969 recall: 0.9174 f1_score: 0.9070 f2_score: 0.9132
16 instances of class Camión with average precision: 0.5130 TP: 7 FP: 2 precision: 0.7778 recall: 0.4375 f1_score: 0.5600 f2_score: 0.4795
7 instances of class Tractor with average precision: 0.3347 TP: 1 FP: 0 precision: 1.0000 recall: 0.1429 f1_score: 0.2500 f2_score: 0.1724
8 instances of class Maquinaria with average precision: 0.6342 TP: 4 FP: 1 precision: 0.8000 recall: 0.5000 f1_score: 0.6154 f2_score: 0.5405
68 instances of class Animal with average precisio

Epoch 36/50
598/598 [==============================] - 440s 736ms/step - loss: 1.1874 - regression_loss: 1.0815 - classification_loss: 0.1059 - val_loss: 1.5782 - val_regression_loss: 1.3714 - val_classification_loss: 0.7846
Running network: 100% (612 of 612) |#####| Elapsed Time: 0:00:51 Time:  0:00:51
Parsing annotations: 100% (612 of 612) |#| Elapsed Time: 0:00:00 Time:  0:00:00
218 instances of class Vehículo with average precision: 0.9330 TP: 200 FP: 26 precision: 0.8850 recall: 0.9174 f1_score: 0.9009 f2_score: 0.9107
16 instances of class Camión with average precision: 0.5275 TP: 7 FP: 1 precision: 0.8750 recall: 0.4375 f1_score: 0.5833 f2_score: 0.4861
7 instances of class Tractor with average precision: 0.2748 TP: 1 FP: 2 precision: 0.3333 recall: 0.1429 f1_score: 0.2000 f2_score: 0.1613
8 instances of class Maquinaria with average precision: 0.6144 TP: 4 FP: 1 precision: 0.8000 recall: 0.5000 f1_score: 0.6154 f2_score: 0.5405
68 instances of class Animal with average precisio

Epoch 41/50
598/598 [==============================] - 436s 730ms/step - loss: 1.1753 - regression_loss: 1.0713 - classification_loss: 0.1039 - val_loss: 1.5935 - val_regression_loss: 1.3649 - val_classification_loss: 0.7643
Running network: 100% (612 of 612) |#####| Elapsed Time: 0:00:52 Time:  0:00:52
Parsing annotations: 100% (612 of 612) |#| Elapsed Time: 0:00:00 Time:  0:00:00
218 instances of class Vehículo with average precision: 0.9252 TP: 200 FP: 25 precision: 0.8889 recall: 0.9174 f1_score: 0.9029 f2_score: 0.9116
16 instances of class Camión with average precision: 0.5351 TP: 6 FP: 2 precision: 0.7500 recall: 0.3750 f1_score: 0.5000 f2_score: 0.4167
7 instances of class Tractor with average precision: 0.3529 TP: 1 FP: 2 precision: 0.3333 recall: 0.1429 f1_score: 0.2000 f2_score: 0.1613
8 instances of class Maquinaria with average precision: 0.6443 TP: 4 FP: 1 precision: 0.8000 recall: 0.5000 f1_score: 0.6154 f2_score: 0.5405
68 instances of class Animal with average precisio

Epoch 46/50
598/598 [==============================] - 428s 716ms/step - loss: 1.1708 - regression_loss: 1.0673 - classification_loss: 0.1035 - val_loss: 1.6516 - val_regression_loss: 1.3985 - val_classification_loss: 0.6501
Running network: 100% (612 of 612) |#####| Elapsed Time: 0:00:52 Time:  0:00:52
Parsing annotations: 100% (612 of 612) |#| Elapsed Time: 0:00:00 Time:  0:00:00
218 instances of class Vehículo with average precision: 0.9302 TP: 200 FP: 25 precision: 0.8889 recall: 0.9174 f1_score: 0.9029 f2_score: 0.9116
16 instances of class Camión with average precision: 0.5183 TP: 7 FP: 2 precision: 0.7778 recall: 0.4375 f1_score: 0.5600 f2_score: 0.4795
7 instances of class Tractor with average precision: 0.5105 TP: 2 FP: 1 precision: 0.6667 recall: 0.2857 f1_score: 0.4000 f2_score: 0.3226
8 instances of class Maquinaria with average precision: 0.6274 TP: 4 FP: 2 precision: 0.6667 recall: 0.5000 f1_score: 0.5714 f2_score: 0.5263
68 instances of class Animal with average precisio


Epoch 00050: ReduceLROnPlateau reducing learning rate to 9.999999974752428e-08.


## Full model again

Changing Loss function to: cls_loss = focal_weight * f2_loss(labels, classification)

In [61]:
BATCH_SIZE = 4

# compute number of annotations to compute number of steps per epoch
df = pd.read_csv(TRAIN_ANNOTATIONS, header=None, names=['img_path', 'x1', 'y1', 'x2', 'y2', 'class'])

countAnn = len(df)-1
countImg = df['img_path'].nunique()
no_steps = math.ceil(countImg/BATCH_SIZE)

print("Count of images: {}".format(countImg))
print("Count of annotations: {}".format(countAnn))
print("Number of steps per epoch: {}".format(no_steps))

Count of images: 4779
Count of annotations: 20574
Number of steps per epoch: 1195


In [62]:
epoch = 35
last_model = os.path.join(SNAPSHOTS_DIR, "resnet50_csv_{}.h5".format(epoch))

!python ../../keras-retinanet/keras_retinanet/bin/train.py \
--config {CONFIG_FILE} \
--random-transform \
--weights {last_model} \
--initial-epoch {epoch} \
--weighted-average \
--batch-size {BATCH_SIZE} \
--steps {no_steps} \
--epochs 100 \
--no-resize \
--image-min-side 1000 \
--image-max-side 1000 \
--compute-val-loss \
--multiprocessing \
--workers 2 \
--tensorboard-dir {TENSORBOARD_LOGS_DIR} \
--snapshot-path {SNAPSHOTS_DIR} \
--validation-freq 5 \
--lr 1e-4 \
--reduce-lr-patience 2 \
--reduce-lr-factor 0.1 \
--gamma 3.0 \
--class-names '/home/aikauel/enap/aerialnet_project/datasets/data/v4/CLASSES' \
csv {TRAIN_ANNOTATIONS} {CLASSES_FILE} \
--val-annotations {VAL_ANNOTATIONS}

Using TensorFlow backend.
Creating model, this may take a second...
2020-09-15 08:27:39.126668: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-09-15 08:27:39.154770: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-09-15 08:27:39.155074: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.755GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2020-09-15 08:27:39.156504: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-09-15 08:27:39.160189: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.s

tracking <tf.Variable 'Variable:0' shape=(15, 4) dtype=float32, numpy=
array([[ -66.944   ,  -16.736   ,   66.944   ,   16.736   ],
       [-105.984   ,  -26.496   ,  105.984   ,   26.496   ],
       [-169.728   ,  -42.432   ,  169.728   ,   42.432   ],
       [ -43.466465,  -25.775614,   43.466465,   25.775614],
       [ -68.81498 ,  -40.80728 ,   68.81498 ,   40.80728 ],
       [-110.20369 ,  -65.35079 ,  110.20369 ,   65.35079 ],
       [ -33.472   ,  -33.472   ,   33.472   ,   33.472   ],
       [ -52.992   ,  -52.992   ,   52.992   ,   52.992   ],
       [ -84.864   ,  -84.864   ,   84.864   ,   84.864   ],
       [ -25.778217,  -43.462074,   25.778217,   43.462074],
       [ -40.811405,  -68.80803 ,   40.811405,   68.80803 ],
       [ -65.35739 , -110.19256 ,   65.35739 ,  110.19256 ],
       [ -16.736   ,  -66.944   ,   16.736   ,   66.944   ],
       [ -26.496   , -105.984   ,   26.496   ,  105.984   ],
       [ -42.432   , -169.728   ,   42.432   ,  169.728   ]],
      dtype=f

Epoch 36/100
2020-09-15 08:27:54.904683: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2020-09-15 08:27:55.853552: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
   1/1195 [..............................] - ETA: 4:21:27 - loss: 1.1485 - regression_loss: 1.0504 - classification_loss: 0.09812020-09-15 08:27:59.226601: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session started.
2020-09-15 08:27:59.226637: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1408] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI could not be loaded or symbol could not be found.
2020-09-15 08:27:59.226646: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1447] function cupti_interface_->ActivityRegisterCallbacks( AllocCuptiActivityBuffer, FreeCuptiActivityBuffer)failed with err


Epoch 00038: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
Epoch 39/100
1195/1195 [==============================] - 624s 522ms/step - loss: 1.3942 - regression_loss: 1.2670 - classification_loss: 0.1271
Running network: 100% (612 of 612) |#####| Elapsed Time: 0:00:50 Time:  0:00:50
Parsing annotations: 100% (612 of 612) |#| Elapsed Time: 0:00:00 Time:  0:00:00
218 instances of class Vehículo with average precision: 0.9099 TP: 189 FP: 30 precision: 0.8630 recall: 0.8670 f1_score: 0.8650 f2_score: 0.8662
16 instances of class Camión with average precision: 0.2711 TP: 3 FP: 1 precision: 0.7500 recall: 0.1875 f1_score: 0.3000 f2_score: 0.2206
7 instances of class Tractor with average precision: 0.1994 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
8 instances of class Maquinaria with average precision: 0.6791 TP: 3 FP: 0 precision: 1.0000 recall: 0.3750 f1_score: 0.5455 f2_score: 0.4286
68 instances of class Animal with average precisi

Epoch 44/100
1195/1195 [==============================] - 629s 526ms/step - loss: 1.2532 - regression_loss: 1.1428 - classification_loss: 0.1104 - val_loss: 1.7349 - val_regression_loss: 1.0577 - val_classification_loss: 0.4534
Running network: 100% (612 of 612) |#####| Elapsed Time: 0:00:51 Time:  0:00:51
Parsing annotations: 100% (612 of 612) |#| Elapsed Time: 0:00:00 Time:  0:00:00
218 instances of class Vehículo with average precision: 0.9166 TP: 193 FP: 24 precision: 0.8894 recall: 0.8853 f1_score: 0.8874 f2_score: 0.8861
16 instances of class Camión with average precision: 0.3786 TP: 4 FP: 1 precision: 0.8000 recall: 0.2500 f1_score: 0.3810 f2_score: 0.2899
7 instances of class Tractor with average precision: 0.3045 TP: 1 FP: 0 precision: 1.0000 recall: 0.1429 f1_score: 0.2500 f2_score: 0.1724
8 instances of class Maquinaria with average precision: 0.5795 TP: 3 FP: 2 precision: 0.6000 recall: 0.3750 f1_score: 0.4615 f2_score: 0.4054
68 instances of class Animal with average preci

...

Epoch 59/100
1195/1195 [==============================] - 611s 511ms/step - loss: 1.1240 - regression_loss: 1.0278 - classification_loss: 0.0963 - val_loss: 1.5622 - val_regression_loss: 1.0497 - val_classification_loss: 0.5448
Running network: 100% (612 of 612) |#####| Elapsed Time: 0:00:51 Time:  0:00:51
Parsing annotations: 100% (612 of 612) |#| Elapsed Time: 0:00:00 Time:  0:00:00
218 instances of class Vehículo with average precision: 0.9391 TP: 198 FP: 24 precision: 0.8919 recall: 0.9083 f1_score: 0.9000 f2_score: 0.9049
16 instances of class Camión with average precision: 0.5311 TP: 7 FP: 1 precision: 0.8750 recall: 0.4375 f1_score: 0.5833 f2_score: 0.4861
7 instances of class Tractor with average precision: 0.3560 TP: 1 FP: 2 precision: 0.3333 recall: 0.1429 f1_score: 0.2000 f2_score: 0.1613
8 instances of class Maquinaria with average precision: 0.7733 TP: 5 FP: 0 precision: 1.0000 recall: 0.6250 f1_score: 0.7692 f2_score: 0.6757
68 instances of class Animal with average preci

Epoch 64/100
1195/1195 [==============================] - 628s 526ms/step - loss: 1.0871 - regression_loss: 0.9934 - classification_loss: 0.0938 - val_loss: 1.5372 - val_regression_loss: 1.0572 - val_classification_loss: 0.6471
Running network: 100% (612 of 612) |#####| Elapsed Time: 0:00:51 Time:  0:00:51
Parsing annotations: 100% (612 of 612) |#| Elapsed Time: 0:00:00 Time:  0:00:00
218 instances of class Vehículo with average precision: 0.9377 TP: 198 FP: 23 precision: 0.8959 recall: 0.9083 f1_score: 0.9021 f2_score: 0.9058
16 instances of class Camión with average precision: 0.5221 TP: 7 FP: 3 precision: 0.7000 recall: 0.4375 f1_score: 0.5385 f2_score: 0.4730
7 instances of class Tractor with average precision: 0.3554 TP: 1 FP: 3 precision: 0.2500 recall: 0.1429 f1_score: 0.1818 f2_score: 0.1562
8 instances of class Maquinaria with average precision: 0.7171 TP: 5 FP: 0 precision: 1.0000 recall: 0.6250 f1_score: 0.7692 f2_score: 0.6757
68 instances of class Animal with average preci

Epoch 69/100
1195/1195 [==============================] - 611s 511ms/step - loss: 1.0605 - regression_loss: 0.9706 - classification_loss: 0.0899 - val_loss: 1.6661 - val_regression_loss: 1.0899 - val_classification_loss: 0.8625
Running network: 100% (612 of 612) |#####| Elapsed Time: 0:00:50 Time:  0:00:50
Parsing annotations: 100% (612 of 612) |#| Elapsed Time: 0:00:00 Time:  0:00:00
218 instances of class Vehículo with average precision: 0.9545 TP: 201 FP: 25 precision: 0.8894 recall: 0.9220 f1_score: 0.9054 f2_score: 0.9153
16 instances of class Camión with average precision: 0.5491 TP: 6 FP: 0 precision: 1.0000 recall: 0.3750 f1_score: 0.5455 f2_score: 0.4286
7 instances of class Tractor with average precision: 0.3314 TP: 1 FP: 3 precision: 0.2500 recall: 0.1429 f1_score: 0.1818 f2_score: 0.1562
8 instances of class Maquinaria with average precision: 0.7292 TP: 4 FP: 0 precision: 1.0000 recall: 0.5000 f1_score: 0.6667 f2_score: 0.5556
68 instances of class Animal with average preci

Epoch 74/100
1195/1195 [==============================] - 627s 525ms/step - loss: 1.0377 - regression_loss: 0.9501 - classification_loss: 0.0876 - val_loss: 1.6317 - val_regression_loss: 1.0716 - val_classification_loss: 0.6665
Running network: 100% (612 of 612) |#####| Elapsed Time: 0:00:51 Time:  0:00:51
Parsing annotations: 100% (612 of 612) |#| Elapsed Time: 0:00:00 Time:  0:00:00
218 instances of class Vehículo with average precision: 0.9458 TP: 204 FP: 30 precision: 0.8718 recall: 0.9358 f1_score: 0.9027 f2_score: 0.9222
16 instances of class Camión with average precision: 0.4798 TP: 7 FP: 2 precision: 0.7778 recall: 0.4375 f1_score: 0.5600 f2_score: 0.4795
7 instances of class Tractor with average precision: 0.3758 TP: 2 FP: 4 precision: 0.3333 recall: 0.2857 f1_score: 0.3077 f2_score: 0.2941
8 instances of class Maquinaria with average precision: 0.8483 TP: 6 FP: 1 precision: 0.8571 recall: 0.7500 f1_score: 0.8000 f2_score: 0.7692
68 instances of class Animal with average preci

8 instances of class Maquinaria with average precision: 0.8041 TP: 5 FP: 3 precision: 0.6250 recall: 0.6250 f1_score: 0.6250 f2_score: 0.6250
68 instances of class Animal with average precision: 0.7498 TP: 49 FP: 11 precision: 0.8167 recall: 0.7206 f1_score: 0.7656 f2_score: 0.7380
11 instances of class Tuberia with average precision: 0.1013 TP: 1 FP: 6 precision: 0.1429 recall: 0.0909 f1_score: 0.1111 f2_score: 0.0980
375 instances of class Escombro with average precision: 0.2520 TP: 83 FP: 129 precision: 0.3915 recall: 0.2213 f1_score: 0.2828 f2_score: 0.2424
8 instances of class Juegos with average precision: 0.5760 TP: 3 FP: 0 precision: 1.0000 recall: 0.3750 f1_score: 0.5455 f2_score: 0.4286
155 instances of class EstrucMetal with average precision: 0.3348 TP: 54 FP: 61 precision: 0.4696 recall: 0.3484 f1_score: 0.4000 f2_score: 0.3673
93 instances of class PalletCaja with average precision: 0.3416 TP: 32 FP: 49 precision: 0.3951 recall: 0.3441 f1_score: 0.3678 f2_score: 0.3532
mA

## Full model training with frequency based weights for focal loss

focal_weight = focal_weight * [1/3367, 1/658, 1, 100, 1/2843, 1/1121, 1/6796, 1, 1, 1]

In [92]:
BATCH_SIZE = 4
TRAIN_ANNOTATIONS = os.path.join(DATASET_VERSION_DIR, 'train.csv')
# compute number of annotations to compute number of steps per epoch
df = pd.read_csv(TRAIN_ANNOTATIONS, header=None, names=['img_path', 'x1', 'y1', 'x2', 'y2', 'class'])

countAnn = len(df)-1
countImg = df['img_path'].nunique()
no_steps = math.ceil(countImg/BATCH_SIZE)

print("Count of images: {}".format(countImg))
print("Count of annotations: {}".format(countAnn))
print("Number of steps per epoch: {}".format(no_steps))

Count of images: 4779
Count of annotations: 20574
Number of steps per epoch: 1195


In [93]:
epoch = 75
last_model = os.path.join(SNAPSHOTS_DIR, "resnet50_csv_{}.h5".format(epoch))

!python ../../keras-retinanet/keras_retinanet/bin/train.py \
--config {CONFIG_FILE} \
--random-transform \
--weights {last_model} \
--initial-epoch {epoch} \
--weighted-average \
--batch-size {BATCH_SIZE} \
--steps {no_steps} \
--epochs 100 \
--no-resize \
--image-min-side 1000 \
--image-max-side 1000 \
--compute-val-loss \
--multiprocessing \
--workers 2 \
--tensorboard-dir {TENSORBOARD_LOGS_DIR} \
--snapshot-path {SNAPSHOTS_DIR} \
--validation-freq 5 \
--lr 1e-4 \
--reduce-lr-patience 2 \
--reduce-lr-factor 0.1 \
--gamma 2.0 \
--class-names '/home/aikauel/enap/aerialnet_project/datasets/data/v4/CLASSES' \
csv {TRAIN_ANNOTATIONS} {CLASSES_FILE} \
--val-annotations {VAL_ANNOTATIONS}

Using TensorFlow backend.
Creating model, this may take a second...
2020-09-15 18:22:24.667438: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-09-15 18:22:24.703955: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-09-15 18:22:24.704317: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.755GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2020-09-15 18:22:24.704496: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-09-15 18:22:24.705397: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.s

tracking <tf.Variable 'Variable:0' shape=(15, 4) dtype=float32, numpy=
array([[-133.888   ,  -33.472   ,  133.888   ,   33.472   ],
       [-211.968   ,  -52.992   ,  211.968   ,   52.992   ],
       [-339.456   ,  -84.864   ,  339.456   ,   84.864   ],
       [ -86.93293 ,  -51.551228,   86.93293 ,   51.551228],
       [-137.62996 ,  -81.61456 ,  137.62996 ,   81.61456 ],
       [-220.40738 , -130.70158 ,  220.40738 ,  130.70158 ],
       [ -66.944   ,  -66.944   ,   66.944   ,   66.944   ],
       [-105.984   , -105.984   ,  105.984   ,  105.984   ],
       [-169.728   , -169.728   ,  169.728   ,  169.728   ],
       [ -51.556435,  -86.92415 ,   51.556435,   86.92415 ],
       [ -81.62281 , -137.61606 ,   81.62281 ,  137.61606 ],
       [-130.71478 , -220.38512 ,  130.71478 ,  220.38512 ],
       [ -33.472   , -133.888   ,   33.472   ,  133.888   ],
       [ -52.992   , -211.968   ,   52.992   ,  211.968   ],
       [ -84.864   , -339.456   ,   84.864   ,  339.456   ]],
      dtype=f

Epoch 76/100
2020-09-15 18:22:40.318110: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2020-09-15 18:22:41.231304: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
   1/1195 [..............................] - ETA: 4:21:27 - loss: 1.0258 - regression_loss: 1.0186 - classification_loss: 0.00722020-09-15 18:22:44.643012: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session started.
2020-09-15 18:22:44.643046: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1408] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI could not be loaded or symbol could not be found.
2020-09-15 18:22:44.643055: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1447] function cupti_interface_->ActivityRegisterCallbacks( AllocCuptiActivityBuffer, FreeCuptiActivityBuffer)failed with err


Epoch 00078: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
Epoch 79/100
1195/1195 [==============================] - 616s 516ms/step - loss: 1.4149 - regression_loss: 1.1767 - classification_loss: 0.2381
Running network: 100% (612 of 612) |#####| Elapsed Time: 0:00:50 Time:  0:00:50
Parsing annotations: 100% (612 of 612) |#| Elapsed Time: 0:00:00 Time:  0:00:00
218 instances of class Vehículo with average precision: 0.8661 TP: 195 FP: 94 precision: 0.6747 recall: 0.8945 f1_score: 0.7692 f2_score: 0.8398
16 instances of class Camión with average precision: 0.0511 TP: 3 FP: 8 precision: 0.2727 recall: 0.1875 f1_score: 0.2222 f2_score: 0.2000
7 instances of class Tractor with average precision: 0.1786 TP: 2 FP: 20 precision: 0.0909 recall: 0.2857 f1_score: 0.1379 f2_score: 0.2000
8 instances of class Maquinaria with average precision: 0.8295 TP: 7 FP: 5 precision: 0.5833 recall: 0.8750 f1_score: 0.7000 f2_score: 0.7955
68 instances of class Animal with average precis

Epoch 84/100
1195/1195 [==============================] - 622s 520ms/step - loss: 1.1879 - regression_loss: 1.0347 - classification_loss: 0.1532 - val_loss: 1.5644 - val_regression_loss: 1.0676 - val_classification_loss: 0.5423
Running network: 100% (612 of 612) |#####| Elapsed Time: 0:00:50 Time:  0:00:50
Parsing annotations: 100% (612 of 612) |#| Elapsed Time: 0:00:00 Time:  0:00:00
218 instances of class Vehículo with average precision: 0.9215 TP: 206 FP: 93 precision: 0.6890 recall: 0.9450 f1_score: 0.7969 f2_score: 0.8796
16 instances of class Camión with average precision: 0.1112 TP: 5 FP: 12 precision: 0.2941 recall: 0.3125 f1_score: 0.3030 f2_score: 0.3086
7 instances of class Tractor with average precision: 0.2857 TP: 3 FP: 26 precision: 0.1034 recall: 0.4286 f1_score: 0.1667 f2_score: 0.2632
8 instances of class Maquinaria with average precision: 0.8750 TP: 7 FP: 3 precision: 0.7000 recall: 0.8750 f1_score: 0.7778 f2_score: 0.8333
68 instances of class Animal with average pre

## Reset full model training with frequency based weights for focal loss -> Camión y Tubería se dejan en 1 (no se ponderan según frecuencia)

focal_weight = focal_weight * [1/3367, 1, 1, 100, 1/2843, 1, 1/6796, 1, 1, 1]

In [98]:
BATCH_SIZE = 4
TRAIN_ANNOTATIONS = os.path.join(DATASET_VERSION_DIR, 'train.csv')
# compute number of annotations to compute number of steps per epoch
df = pd.read_csv(TRAIN_ANNOTATIONS, header=None, names=['img_path', 'x1', 'y1', 'x2', 'y2', 'class'])

countAnn = len(df)-1
countImg = df['img_path'].nunique()
no_steps = math.ceil(countImg/BATCH_SIZE)

print("Count of images: {}".format(countImg))
print("Count of annotations: {}".format(countAnn))
print("Number of steps per epoch: {}".format(no_steps))

Count of images: 4779
Count of annotations: 20574
Number of steps per epoch: 1195


In [99]:
epoch = 75 # start from same point
last_model = os.path.join(SNAPSHOTS_DIR, "resnet50_csv_{}.h5".format(epoch))

!python ../../keras-retinanet/keras_retinanet/bin/train.py \
--config {CONFIG_FILE} \
--random-transform \
--weights {last_model} \
--initial-epoch {epoch} \
--weighted-average \
--batch-size {BATCH_SIZE} \
--steps {no_steps} \
--epochs 100 \
--no-resize \
--image-min-side 1000 \
--image-max-side 1000 \
--compute-val-loss \
--multiprocessing \
--workers 2 \
--tensorboard-dir {TENSORBOARD_LOGS_DIR} \
--snapshot-path {SNAPSHOTS_DIR} \
--validation-freq 5 \
--lr 0.5e-4 \
--reduce-lr-patience 2 \
--reduce-lr-factor 0.1 \
--gamma 2.0 \
--class-names '/home/aikauel/enap/aerialnet_project/datasets/data/v4/CLASSES' \
csv {TRAIN_ANNOTATIONS} {CLASSES_FILE} \
--val-annotations {VAL_ANNOTATIONS}

Using TensorFlow backend.
Creating model, this may take a second...
2020-09-15 20:41:01.193974: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-09-15 20:41:01.222541: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-09-15 20:41:01.222867: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.755GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2020-09-15 20:41:01.223030: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-09-15 20:41:01.224084: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.s

tracking <tf.Variable 'Variable:0' shape=(15, 4) dtype=float32, numpy=
array([[ -66.944   ,  -16.736   ,   66.944   ,   16.736   ],
       [-105.984   ,  -26.496   ,  105.984   ,   26.496   ],
       [-169.728   ,  -42.432   ,  169.728   ,   42.432   ],
       [ -43.466465,  -25.775614,   43.466465,   25.775614],
       [ -68.81498 ,  -40.80728 ,   68.81498 ,   40.80728 ],
       [-110.20369 ,  -65.35079 ,  110.20369 ,   65.35079 ],
       [ -33.472   ,  -33.472   ,   33.472   ,   33.472   ],
       [ -52.992   ,  -52.992   ,   52.992   ,   52.992   ],
       [ -84.864   ,  -84.864   ,   84.864   ,   84.864   ],
       [ -25.778217,  -43.462074,   25.778217,   43.462074],
       [ -40.811405,  -68.80803 ,   40.811405,   68.80803 ],
       [ -65.35739 , -110.19256 ,   65.35739 ,  110.19256 ],
       [ -16.736   ,  -66.944   ,   16.736   ,   66.944   ],
       [ -26.496   , -105.984   ,   26.496   ,  105.984   ],
       [ -42.432   , -169.728   ,   42.432   ,  169.728   ]],
      dtype=f

Epoch 76/100
2020-09-15 20:41:16.723864: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2020-09-15 20:41:17.657355: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2020-09-15 20:41:19.495250: W tensorflow/core/common_runtime/bfc_allocator.cc:245] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.08GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2020-09-15 20:41:19.540735: W tensorflow/core/common_runtime/bfc_allocator.cc:245] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.11GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
   1/1195 [..............................] - ETA: 4:19:49 - loss: 0.9601 - regression_loss: 0


Epoch 00078: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-06.
Epoch 79/100
1195/1195 [==============================] - 611s 511ms/step - loss: 1.2215 - regression_loss: 1.0294 - classification_loss: 0.1920
Running network: 100% (612 of 612) |#####| Elapsed Time: 0:00:51 Time:  0:00:51
Parsing annotations: 100% (612 of 612) |#| Elapsed Time: 0:00:00 Time:  0:00:00
218 instances of class Vehículo with average precision: 0.9224 TP: 207 FP: 125 precision: 0.6235 recall: 0.9495 f1_score: 0.7527 f2_score: 0.8596
16 instances of class Camión with average precision: 0.1944 TP: 8 FP: 32 precision: 0.2000 recall: 0.5000 f1_score: 0.2857 f2_score: 0.3846
7 instances of class Tractor with average precision: 0.3780 TP: 5 FP: 22 precision: 0.1852 recall: 0.7143 f1_score: 0.2941 f2_score: 0.4545
8 instances of class Maquinaria with average precision: 0.8594 TP: 7 FP: 5 precision: 0.5833 recall: 0.8750 f1_score: 0.7000 f2_score: 0.7955
68 instances of class Animal with average prec

Epoch 84/100
1195/1195 [==============================] - 620s 519ms/step - loss: 1.0797 - regression_loss: 0.9407 - classification_loss: 0.1390 - val_loss: 1.5720 - val_regression_loss: 1.0561 - val_classification_loss: 0.5067
Running network: 100% (612 of 612) |#####| Elapsed Time: 0:00:51 Time:  0:00:51
Parsing annotations: 100% (612 of 612) |#| Elapsed Time: 0:00:00 Time:  0:00:00
218 instances of class Vehículo with average precision: 0.9422 TP: 211 FP: 128 precision: 0.6224 recall: 0.9679 f1_score: 0.7576 f2_score: 0.8712
16 instances of class Camión with average precision: 0.3923 TP: 10 FP: 30 precision: 0.2500 recall: 0.6250 f1_score: 0.3571 f2_score: 0.4808
7 instances of class Tractor with average precision: 0.4732 TP: 5 FP: 26 precision: 0.1613 recall: 0.7143 f1_score: 0.2632 f2_score: 0.4237
8 instances of class Maquinaria with average precision: 0.7500 TP: 6 FP: 6 precision: 0.5000 recall: 0.7500 f1_score: 0.6000 f2_score: 0.6818
68 instances of class Animal with average p

Epoch 89/100
1195/1195 [==============================] - 629s 526ms/step - loss: 1.0272 - regression_loss: 0.9086 - classification_loss: 0.1186 - val_loss: 1.4712 - val_regression_loss: 1.0245 - val_classification_loss: 0.5627
Running network: 100% (612 of 612) |#####| Elapsed Time: 0:00:51 Time:  0:00:51
Parsing annotations: 100% (612 of 612) |#| Elapsed Time: 0:00:00 Time:  0:00:00
218 instances of class Vehículo with average precision: 0.9420 TP: 211 FP: 150 precision: 0.5845 recall: 0.9679 f1_score: 0.7288 f2_score: 0.8556
16 instances of class Camión with average precision: 0.4385 TP: 10 FP: 26 precision: 0.2778 recall: 0.6250 f1_score: 0.3846 f2_score: 0.5000
7 instances of class Tractor with average precision: 0.4650 TP: 6 FP: 24 precision: 0.2000 recall: 0.8571 f1_score: 0.3243 f2_score: 0.5172
8 instances of class Maquinaria with average precision: 0.8594 TP: 7 FP: 2 precision: 0.7778 recall: 0.8750 f1_score: 0.8235 f2_score: 0.8537
68 instances of class Animal with average p

Epoch 94/100
1195/1195 [==============================] - 622s 520ms/step - loss: 1.0160 - regression_loss: 0.8913 - classification_loss: 0.1247 - val_loss: 1.5156 - val_regression_loss: 1.0635 - val_classification_loss: 0.6293
Running network: 100% (612 of 612) |#####| Elapsed Time: 0:00:51 Time:  0:00:51
Parsing annotations: 100% (612 of 612) |#| Elapsed Time: 0:00:00 Time:  0:00:00
218 instances of class Vehículo with average precision: 0.9427 TP: 210 FP: 148 precision: 0.5866 recall: 0.9633 f1_score: 0.7292 f2_score: 0.8537
16 instances of class Camión with average precision: 0.5397 TP: 14 FP: 52 precision: 0.2121 recall: 0.8750 f1_score: 0.3415 f2_score: 0.5385
7 instances of class Tractor with average precision: 0.5373 TP: 7 FP: 33 precision: 0.1750 recall: 1.0000 f1_score: 0.2979 f2_score: 0.5147
8 instances of class Maquinaria with average precision: 0.8173 TP: 7 FP: 8 precision: 0.4667 recall: 0.8750 f1_score: 0.6087 f2_score: 0.7447
68 instances of class Animal with average p

Epoch 99/100
1195/1195 [==============================] - 629s 526ms/step - loss: 0.9830 - regression_loss: 0.8749 - classification_loss: 0.1081 - val_loss: 1.5567 - val_regression_loss: 1.0519 - val_classification_loss: 0.8804
Running network: 100% (612 of 612) |#####| Elapsed Time: 0:00:51 Time:  0:00:51
Parsing annotations: 100% (612 of 612) |#| Elapsed Time: 0:00:00 Time:  0:00:00
218 instances of class Vehículo with average precision: 0.9360 TP: 209 FP: 131 precision: 0.6147 recall: 0.9587 f1_score: 0.7491 f2_score: 0.8622
16 instances of class Camión with average precision: 0.4642 TP: 11 FP: 32 precision: 0.2558 recall: 0.6875 f1_score: 0.3729 f2_score: 0.5140
7 instances of class Tractor with average precision: 0.6270 TP: 7 FP: 28 precision: 0.2000 recall: 1.0000 f1_score: 0.3333 f2_score: 0.5556
8 instances of class Maquinaria with average precision: 0.8229 TP: 7 FP: 7 precision: 0.5000 recall: 0.8750 f1_score: 0.6364 f2_score: 0.7609
68 instances of class Animal with average p

# Iteration 2: reset from 1st epoch

focal_weight = focal_weight * [1/3367, 1, 1, 10, 1, 1, 1, 1, 1, 1]\
cls_loss = focal_weight * keras.backend.binary_crossentropy(labels, classification)

### FCNs training (regression, classification) with freezed backbone  

First, train the model freezing the backbone. The backbone weights will be the weights from the previous model train before.
Set gamma with high value (>2.0) to weight the loss of well-classified classes down, forcing the model to learn on harder (in this case: less examples) classes.

In [116]:
BATCH_SIZE = 8
TRAIN_ANNOTATIONS = os.path.join(DATASET_VERSION_DIR, 'train.csv')
# compute number of annotations to compute number of steps per epoch
df = pd.read_csv(TRAIN_ANNOTATIONS, header=None, names=['img_path', 'x1', 'y1', 'x2', 'y2', 'class'])

countAnn = len(df)-1
countImg = df['img_path'].nunique()
no_steps = math.ceil(countImg/BATCH_SIZE)

print("Count of images: {}".format(countImg))
print("Count of annotations: {}".format(countAnn))
print("Number of steps per epoch: {}".format(no_steps))

# Using pretrained model from 11_classes
PRETRAINED_MODEL = '/home/aikauel/enap/aerialnet_project/11_classes/snapshots/resnet50_csv_65.h5'

TENSORBOARD_LOGS_DIR = os.path.join(TRAINING_BASE_DIR, 'logs2')
SNAPSHOTS_DIR = os.path.join(TRAINING_BASE_DIR, 'snapshots2')

Count of images: 4779
Count of annotations: 20574
Number of steps per epoch: 598


In [120]:
!python ../../keras-retinanet/keras_retinanet/bin/train.py \
--config {CONFIG_FILE} \
--freeze-backbone \
--random-transform \
--weights {PRETRAINED_MODEL} \
--weighted-average \
--batch-size {BATCH_SIZE} \
--steps {no_steps} \
--epochs 50 \
--no-resize \
--image-min-side 1000 \
--image-max-side 1000 \
--compute-val-loss \
--multiprocessing \
--workers 2 \
--tensorboard-dir {TENSORBOARD_LOGS_DIR} \
--snapshot-path {SNAPSHOTS_DIR} \
--validation-freq 5 \
--gamma 3.0 \
--class-names '/home/aikauel/enap/aerialnet_project/datasets/data/v4/CLASSES' \
csv {TRAIN_ANNOTATIONS} {CLASSES_FILE} \
--val-annotations {VAL_ANNOTATIONS}

Using TensorFlow backend.
Creating model, this may take a second...
2020-09-16 12:08:46.649483: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-09-16 12:08:46.681026: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-09-16 12:08:46.681342: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.755GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2020-09-16 12:08:46.681490: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-09-16 12:08:46.682989: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.s

tracking <tf.Variable 'Variable:0' shape=(15, 4) dtype=float32, numpy=
array([[ -66.944   ,  -16.736   ,   66.944   ,   16.736   ],
       [-105.984   ,  -26.496   ,  105.984   ,   26.496   ],
       [-169.728   ,  -42.432   ,  169.728   ,   42.432   ],
       [ -43.466465,  -25.775614,   43.466465,   25.775614],
       [ -68.81498 ,  -40.80728 ,   68.81498 ,   40.80728 ],
       [-110.20369 ,  -65.35079 ,  110.20369 ,   65.35079 ],
       [ -33.472   ,  -33.472   ,   33.472   ,   33.472   ],
       [ -52.992   ,  -52.992   ,   52.992   ,   52.992   ],
       [ -84.864   ,  -84.864   ,   84.864   ,   84.864   ],
       [ -25.778217,  -43.462074,   25.778217,   43.462074],
       [ -40.811405,  -68.80803 ,   40.811405,   68.80803 ],
       [ -65.35739 , -110.19256 ,   65.35739 ,  110.19256 ],
       [ -16.736   ,  -66.944   ,   16.736   ,   66.944   ],
       [ -26.496   , -105.984   ,   26.496   ,  105.984   ],
       [ -42.432   , -169.728   ,   42.432   ,  169.728   ]],
      dtype=f

Epoch 1/50
2020-09-16 12:08:56.607520: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2020-09-16 12:08:57.901238: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
  1/598 [..............................] - ETA: 1:42:26 - loss: 167.9532 - regression_loss: 2.0002 - classification_loss: 165.95302020-09-16 12:09:01.248143: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session started.
2020-09-16 12:09:01.248179: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1408] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI could not be loaded or symbol could not be found.
2020-09-16 12:09:01.248186: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1447] function cupti_interface_->ActivityRegisterCallbacks( AllocCuptiActivityBuffer, FreeCuptiActivityBuffer)failed with err

Epoch 4/50
598/598 [==============================] - 420s 702ms/step - loss: 1.8165 - regression_loss: 1.5229 - classification_loss: 0.2936
Running network: 100% (612 of 612) |#####| Elapsed Time: 0:00:55 Time:  0:00:55
Parsing annotations: 100% (612 of 612) |#| Elapsed Time: 0:00:00 Time:  0:00:00
218 instances of class Vehículo with average precision: 0.9143 TP: 214 FP: 2941 precision: 0.0678 recall: 0.9817 f1_score: 0.1269 f2_score: 0.2657
16 instances of class Camión with average precision: 0.3652 TP: 15 FP: 547 precision: 0.0267 recall: 0.9375 f1_score: 0.0519 f2_score: 0.1198
7 instances of class Tractor with average precision: 0.0691 TP: 4 FP: 343 precision: 0.0115 recall: 0.5714 f1_score: 0.0226 f2_score: 0.0533
8 instances of class Maquinaria with average precision: 0.7381 TP: 8 FP: 256 precision: 0.0303 recall: 1.0000 f1_score: 0.0588 f2_score: 0.1351
68 instances of class Animal with average precision: 0.4966 TP: 58 FP: 8185 precision: 0.0070 recall: 0.8529 f1_score: 0.0140

Epoch 9/50
598/598 [==============================] - 438s 733ms/step - loss: 1.7250 - regression_loss: 1.4652 - classification_loss: 0.2598 - val_loss: 2.0023 - val_regression_loss: 1.5153 - val_classification_loss: 1.5797
Running network: 100% (612 of 612) |#####| Elapsed Time: 0:00:54 Time:  0:00:54
Parsing annotations: 100% (612 of 612) |#| Elapsed Time: 0:00:00 Time:  0:00:00
218 instances of class Vehículo with average precision: 0.9295 TP: 215 FP: 2121 precision: 0.0920 recall: 0.9862 f1_score: 0.1684 f2_score: 0.3351
16 instances of class Camión with average precision: 0.3278 TP: 15 FP: 528 precision: 0.0276 recall: 0.9375 f1_score: 0.0537 f2_score: 0.1236
7 instances of class Tractor with average precision: 0.2041 TP: 5 FP: 223 precision: 0.0219 recall: 0.7143 f1_score: 0.0426 f2_score: 0.0977
8 instances of class Maquinaria with average precision: 0.7960 TP: 8 FP: 123 precision: 0.0611 recall: 1.0000 f1_score: 0.1151 f2_score: 0.2454
68 instances of class Animal with average 

Epoch 14/50
598/598 [==============================] - 432s 722ms/step - loss: 1.6762 - regression_loss: 1.4304 - classification_loss: 0.2458 - val_loss: 1.9218 - val_regression_loss: 1.4726 - val_classification_loss: 1.0403
Running network: 100% (612 of 612) |#####| Elapsed Time: 0:00:54 Time:  0:00:54
Parsing annotations: 100% (612 of 612) |#| Elapsed Time: 0:00:00 Time:  0:00:00
218 instances of class Vehículo with average precision: 0.9286 TP: 215 FP: 1974 precision: 0.0982 recall: 0.9862 f1_score: 0.1786 f2_score: 0.3512
16 instances of class Camión with average precision: 0.3834 TP: 15 FP: 540 precision: 0.0270 recall: 0.9375 f1_score: 0.0525 f2_score: 0.1212
7 instances of class Tractor with average precision: 0.3351 TP: 6 FP: 221 precision: 0.0264 recall: 0.8571 f1_score: 0.0513 f2_score: 0.1176
8 instances of class Maquinaria with average precision: 0.7487 TP: 8 FP: 202 precision: 0.0381 recall: 1.0000 f1_score: 0.0734 f2_score: 0.1653
68 instances of class Animal with average

Epoch 19/50
598/598 [==============================] - 443s 740ms/step - loss: 1.6541 - regression_loss: 1.4150 - classification_loss: 0.2390 - val_loss: 1.8494 - val_regression_loss: 1.4558 - val_classification_loss: 2.0258
Running network: 100% (612 of 612) |#####| Elapsed Time: 0:00:54 Time:  0:00:54
Parsing annotations: 100% (612 of 612) |#| Elapsed Time: 0:00:00 Time:  0:00:00
218 instances of class Vehículo with average precision: 0.9114 TP: 213 FP: 1765 precision: 0.1077 recall: 0.9771 f1_score: 0.1940 f2_score: 0.3737
16 instances of class Camión with average precision: 0.4114 TP: 15 FP: 516 precision: 0.0282 recall: 0.9375 f1_score: 0.0548 f2_score: 0.1261
7 instances of class Tractor with average precision: 0.2851 TP: 6 FP: 240 precision: 0.0244 recall: 0.8571 f1_score: 0.0474 f2_score: 0.1095
8 instances of class Maquinaria with average precision: 0.7691 TP: 8 FP: 166 precision: 0.0460 recall: 1.0000 f1_score: 0.0879 f2_score: 0.1942
68 instances of class Animal with average

## Full model training

focal_weight = focal_weight \
cls_loss = focal_weight * keras.backend.binary_crossentropy(labels, classification) * f2_loss(labels, classification)

In [121]:
BATCH_SIZE = 4
TRAIN_ANNOTATIONS = os.path.join(DATASET_VERSION_DIR, 'train.csv')
# compute number of annotations to compute number of steps per epoch
df = pd.read_csv(TRAIN_ANNOTATIONS, header=None, names=['img_path', 'x1', 'y1', 'x2', 'y2', 'class'])

countAnn = len(df)-1
countImg = df['img_path'].nunique()
no_steps = math.ceil(countImg/BATCH_SIZE)

print("Count of images: {}".format(countImg))
print("Count of annotations: {}".format(countAnn))
print("Number of steps per epoch: {}".format(no_steps))

Count of images: 4779
Count of annotations: 20574
Number of steps per epoch: 1195


In [122]:
epoch = 13 # start from same point
last_model = os.path.join(SNAPSHOTS_DIR, "resnet50_csv_{}.h5".format(epoch))

!python ../../keras-retinanet/keras_retinanet/bin/train.py \
--config {CONFIG_FILE} \
--random-transform \
--weights {last_model} \
--initial-epoch {epoch} \
--weighted-average \
--batch-size {BATCH_SIZE} \
--steps {no_steps} \
--epochs 100 \
--no-resize \
--image-min-side 1000 \
--image-max-side 1000 \
--compute-val-loss \
--multiprocessing \
--workers 2 \
--tensorboard-dir {TENSORBOARD_LOGS_DIR} \
--snapshot-path {SNAPSHOTS_DIR} \
--validation-freq 5 \
--lr 0.5e-4 \
--reduce-lr-patience 2 \
--reduce-lr-factor 0.1 \
--gamma 3.0 \
--class-names '/home/aikauel/enap/aerialnet_project/datasets/data/v4/CLASSES' \
csv {TRAIN_ANNOTATIONS} {CLASSES_FILE} \
--val-annotations {VAL_ANNOTATIONS}

Using TensorFlow backend.
Creating model, this may take a second...
2020-09-16 15:18:11.200236: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-09-16 15:18:11.231371: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-09-16 15:18:11.231714: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.755GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2020-09-16 15:18:11.231865: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-09-16 15:18:11.233179: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.s

tracking <tf.Variable 'Variable:0' shape=(15, 4) dtype=float32, numpy=
array([[ -66.944   ,  -16.736   ,   66.944   ,   16.736   ],
       [-105.984   ,  -26.496   ,  105.984   ,   26.496   ],
       [-169.728   ,  -42.432   ,  169.728   ,   42.432   ],
       [ -43.466465,  -25.775614,   43.466465,   25.775614],
       [ -68.81498 ,  -40.80728 ,   68.81498 ,   40.80728 ],
       [-110.20369 ,  -65.35079 ,  110.20369 ,   65.35079 ],
       [ -33.472   ,  -33.472   ,   33.472   ,   33.472   ],
       [ -52.992   ,  -52.992   ,   52.992   ,   52.992   ],
       [ -84.864   ,  -84.864   ,   84.864   ,   84.864   ],
       [ -25.778217,  -43.462074,   25.778217,   43.462074],
       [ -40.811405,  -68.80803 ,   40.811405,   68.80803 ],
       [ -65.35739 , -110.19256 ,   65.35739 ,  110.19256 ],
       [ -16.736   ,  -66.944   ,   16.736   ,   66.944   ],
       [ -26.496   , -105.984   ,   26.496   ,  105.984   ],
       [ -42.432   , -169.728   ,   42.432   ,  169.728   ]],
      dtype=f

Epoch 14/100
2020-09-16 15:18:27.202624: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2020-09-16 15:18:28.142401: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
   1/1195 [..............................] - ETA: 4:24:56 - loss: 2.0344 - regression_loss: 1.7574 - classification_loss: 0.27702020-09-16 15:18:31.592476: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session started.
2020-09-16 15:18:31.592510: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1408] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI could not be loaded or symbol could not be found.
2020-09-16 15:18:31.592520: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1447] function cupti_interface_->ActivityRegisterCallbacks( AllocCuptiActivityBuffer, FreeCuptiActivityBuffer)failed with err

Epoch 17/100
1195/1195 [==============================] - 631s 528ms/step - loss: 1.5885 - regression_loss: 1.3742 - classification_loss: 0.2142 - val_loss: 2.3477 - val_regression_loss: 1.3543 - val_classification_loss: 0.7143
Running network: 100% (612 of 612) |#####| Elapsed Time: 0:00:52 Time:  0:00:52
Parsing annotations: 100% (612 of 612) |#| Elapsed Time: 0:00:00 Time:  0:00:00
218 instances of class Vehículo with average precision: 0.9412 TP: 215 FP: 892 precision: 0.1942 recall: 0.9862 f1_score: 0.3245 f2_score: 0.5432
16 instances of class Camión with average precision: 0.1594 TP: 14 FP: 503 precision: 0.0271 recall: 0.8750 f1_score: 0.0525 f2_score: 0.1205
7 instances of class Tractor with average precision: 0.2905 TP: 3 FP: 223 precision: 0.0133 recall: 0.4286 f1_score: 0.0258 f2_score: 0.0591
8 instances of class Maquinaria with average precision: 0.6886 TP: 8 FP: 254 precision: 0.0305 recall: 1.0000 f1_score: 0.0593 f2_score: 0.1361
68 instances of class Animal with avera

Epoch 22/100
1195/1195 [==============================] - 652s 545ms/step - loss: 1.4860 - regression_loss: 1.2985 - classification_loss: 0.1875 - val_loss: 2.0057 - val_regression_loss: 1.3184 - val_classification_loss: 1.0958
Running network: 100% (612 of 612) |#####| Elapsed Time: 0:00:53 Time:  0:00:53
Parsing annotations: 100% (612 of 612) |#| Elapsed Time: 0:00:00 Time:  0:00:00
218 instances of class Vehículo with average precision: 0.9408 TP: 216 FP: 1248 precision: 0.1475 recall: 0.9908 f1_score: 0.2568 f2_score: 0.4623
16 instances of class Camión with average precision: 0.3608 TP: 15 FP: 996 precision: 0.0148 recall: 0.9375 f1_score: 0.0292 f2_score: 0.0698
7 instances of class Tractor with average precision: 0.5025 TP: 6 FP: 355 precision: 0.0166 recall: 0.8571 f1_score: 0.0326 f2_score: 0.0771
8 instances of class Maquinaria with average precision: 0.6412 TP: 8 FP: 435 precision: 0.0181 recall: 1.0000 f1_score: 0.0355 f2_score: 0.0842
68 instances of class Animal with aver

Epoch 27/100
1195/1195 [==============================] - 630s 527ms/step - loss: 1.3984 - regression_loss: 1.2283 - classification_loss: 0.1701 - val_loss: 1.7638 - val_regression_loss: 1.1164 - val_classification_loss: 0.9541
Running network: 100% (612 of 612) |#####| Elapsed Time: 0:00:53 Time:  0:00:53
Parsing annotations: 100% (612 of 612) |#| Elapsed Time: 0:00:00 Time:  0:00:00
218 instances of class Vehículo with average precision: 0.9411 TP: 213 FP: 468 precision: 0.3128 recall: 0.9771 f1_score: 0.4739 f2_score: 0.6858
16 instances of class Camión with average precision: 0.3705 TP: 13 FP: 630 precision: 0.0202 recall: 0.8125 f1_score: 0.0395 f2_score: 0.0919
7 instances of class Tractor with average precision: 0.2058 TP: 5 FP: 332 precision: 0.0148 recall: 0.7143 f1_score: 0.0291 f2_score: 0.0685
8 instances of class Maquinaria with average precision: 0.8059 TP: 8 FP: 603 precision: 0.0131 recall: 1.0000 f1_score: 0.0258 f2_score: 0.0622
68 instances of class Animal with avera

Epoch 32/100
1195/1195 [==============================] - 632s 529ms/step - loss: 1.3467 - regression_loss: 1.1913 - classification_loss: 0.1553 - val_loss: 1.7631 - val_regression_loss: 1.1812 - val_classification_loss: 0.9499
Running network: 100% (612 of 612) |#####| Elapsed Time: 0:00:53 Time:  0:00:53
Parsing annotations: 100% (612 of 612) |#| Elapsed Time: 0:00:00 Time:  0:00:00
218 instances of class Vehículo with average precision: 0.9351 TP: 210 FP: 351 precision: 0.3743 recall: 0.9633 f1_score: 0.5392 f2_score: 0.7327
16 instances of class Camión with average precision: 0.4725 TP: 15 FP: 709 precision: 0.0207 recall: 0.9375 f1_score: 0.0405 f2_score: 0.0952
7 instances of class Tractor with average precision: 0.1675 TP: 3 FP: 228 precision: 0.0130 recall: 0.4286 f1_score: 0.0252 f2_score: 0.0579
8 instances of class Maquinaria with average precision: 0.6625 TP: 6 FP: 376 precision: 0.0157 recall: 0.7500 f1_score: 0.0308 f2_score: 0.0725
68 instances of class Animal with avera

Epoch 37/100
1195/1195 [==============================] - 643s 538ms/step - loss: 1.2878 - regression_loss: 1.1413 - classification_loss: 0.1465 - val_loss: 1.6248 - val_regression_loss: 1.0986 - val_classification_loss: 0.7686
Running network: 100% (612 of 612) |#####| Elapsed Time: 0:00:52 Time:  0:00:52
Parsing annotations: 100% (612 of 612) |#| Elapsed Time: 0:00:00 Time:  0:00:00
218 instances of class Vehículo with average precision: 0.9500 TP: 214 FP: 418 precision: 0.3386 recall: 0.9817 f1_score: 0.5035 f2_score: 0.7114
16 instances of class Camión with average precision: 0.4492 TP: 15 FP: 644 precision: 0.0228 recall: 0.9375 f1_score: 0.0444 f2_score: 0.1037
7 instances of class Tractor with average precision: 0.1637 TP: 4 FP: 477 precision: 0.0083 recall: 0.5714 f1_score: 0.0164 f2_score: 0.0393
8 instances of class Maquinaria with average precision: 0.6062 TP: 8 FP: 419 precision: 0.0187 recall: 1.0000 f1_score: 0.0368 f2_score: 0.0871
68 instances of class Animal with avera

Epoch 42/100
1195/1195 [==============================] - 637s 533ms/step - loss: 1.2395 - regression_loss: 1.1030 - classification_loss: 0.1365 - val_loss: 1.7059 - val_regression_loss: 1.0912 - val_classification_loss: 1.1856
Running network: 100% (612 of 612) |#####| Elapsed Time: 0:00:52 Time:  0:00:52
Parsing annotations: 100% (612 of 612) |#| Elapsed Time: 0:00:00 Time:  0:00:00
218 instances of class Vehículo with average precision: 0.9308 TP: 209 FP: 626 precision: 0.2503 recall: 0.9587 f1_score: 0.3970 f2_score: 0.6122
16 instances of class Camión with average precision: 0.3176 TP: 15 FP: 949 precision: 0.0156 recall: 0.9375 f1_score: 0.0306 f2_score: 0.0730
7 instances of class Tractor with average precision: 0.2015 TP: 6 FP: 646 precision: 0.0092 recall: 0.8571 f1_score: 0.0182 f2_score: 0.0441
8 instances of class Maquinaria with average precision: 0.7179 TP: 8 FP: 495 precision: 0.0159 recall: 1.0000 f1_score: 0.0313 f2_score: 0.0748
68 instances of class Animal with avera

Epoch 47/100
1195/1195 [==============================] - 618s 517ms/step - loss: 1.1869 - regression_loss: 1.0581 - classification_loss: 0.1288 - val_loss: 1.5984 - val_regression_loss: 1.1192 - val_classification_loss: 0.9896
Running network: 100% (612 of 612) |#####| Elapsed Time: 0:00:52 Time:  0:00:52
Parsing annotations: 100% (612 of 612) |#| Elapsed Time: 0:00:00 Time:  0:00:00
218 instances of class Vehículo with average precision: 0.9575 TP: 216 FP: 801 precision: 0.2124 recall: 0.9908 f1_score: 0.3498 f2_score: 0.5717
16 instances of class Camión with average precision: 0.4157 TP: 15 FP: 983 precision: 0.0150 recall: 0.9375 f1_score: 0.0296 f2_score: 0.0706
7 instances of class Tractor with average precision: 0.3409 TP: 5 FP: 428 precision: 0.0115 recall: 0.7143 f1_score: 0.0227 f2_score: 0.0542
8 instances of class Maquinaria with average precision: 0.7304 TP: 8 FP: 494 precision: 0.0159 recall: 1.0000 f1_score: 0.0314 f2_score: 0.0749
68 instances of class Animal with avera

## Full model training

focal_weight = focal_weight = focal_weight * [1/10, 1, 1, 100, 1, 1, 1, 1, 1, 1] \
cls_loss = focal_weight * keras.backend.binary_crossentropy(labels, classification) * f2_loss(labels, classification)

In [136]:
BATCH_SIZE = 4
TRAIN_ANNOTATIONS = os.path.join(DATASET_VERSION_DIR, 'train.csv')
# compute number of annotations to compute number of steps per epoch
df = pd.read_csv(TRAIN_ANNOTATIONS, header=None, names=['img_path', 'x1', 'y1', 'x2', 'y2', 'class'])

countAnn = len(df)-1
countImg = df['img_path'].nunique()
no_steps = math.ceil(countImg/BATCH_SIZE)

print("Count of images: {}".format(countImg))
print("Count of annotations: {}".format(countAnn))
print("Number of steps per epoch: {}".format(no_steps))

Count of images: 4779
Count of annotations: 20574
Number of steps per epoch: 1195


In [137]:
epoch = 46 # start from same point
last_model = os.path.join(SNAPSHOTS_DIR, "resnet50_csv_{}.h5".format(epoch))

!python ../../keras-retinanet/keras_retinanet/bin/train.py \
--config {CONFIG_FILE} \
--random-transform \
--weights {last_model} \
--initial-epoch {epoch} \
--weighted-average \
--batch-size {BATCH_SIZE} \
--steps {no_steps} \
--epochs 100 \
--no-resize \
--image-min-side 1000 \
--image-max-side 1000 \
--compute-val-loss \
--multiprocessing \
--workers 2 \
--tensorboard-dir {TENSORBOARD_LOGS_DIR} \
--snapshot-path {SNAPSHOTS_DIR} \
--validation-freq 5 \
--lr 0.5e-4 \
--reduce-lr-patience 2 \
--reduce-lr-factor 0.1 \
--gamma 3.0 \
--class-names '/home/aikauel/enap/aerialnet_project/datasets/data/v4/CLASSES' \
csv {TRAIN_ANNOTATIONS} {CLASSES_FILE} \
--val-annotations {VAL_ANNOTATIONS}

Using TensorFlow backend.
Creating model, this may take a second...
2020-09-16 22:31:04.545476: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-09-16 22:31:04.576280: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-09-16 22:31:04.576612: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.755GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2020-09-16 22:31:04.576768: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-09-16 22:31:04.577760: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.s

tracking <tf.Variable 'Variable:0' shape=(15, 4) dtype=float32, numpy=
array([[ -66.944   ,  -16.736   ,   66.944   ,   16.736   ],
       [-105.984   ,  -26.496   ,  105.984   ,   26.496   ],
       [-169.728   ,  -42.432   ,  169.728   ,   42.432   ],
       [ -43.466465,  -25.775614,   43.466465,   25.775614],
       [ -68.81498 ,  -40.80728 ,   68.81498 ,   40.80728 ],
       [-110.20369 ,  -65.35079 ,  110.20369 ,   65.35079 ],
       [ -33.472   ,  -33.472   ,   33.472   ,   33.472   ],
       [ -52.992   ,  -52.992   ,   52.992   ,   52.992   ],
       [ -84.864   ,  -84.864   ,   84.864   ,   84.864   ],
       [ -25.778217,  -43.462074,   25.778217,   43.462074],
       [ -40.811405,  -68.80803 ,   40.811405,   68.80803 ],
       [ -65.35739 , -110.19256 ,   65.35739 ,  110.19256 ],
       [ -16.736   ,  -66.944   ,   16.736   ,   66.944   ],
       [ -26.496   , -105.984   ,   26.496   ,  105.984   ],
       [ -42.432   , -169.728   ,   42.432   ,  169.728   ]],
      dtype=f

2020-09-16 22:31:06.139890: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcupti.so.10.1'; dlerror: libcupti.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/x86_64-linux-gnu
2020-09-16 22:31:06.139914: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1408] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI could not be loaded or symbol could not be found.
2020-09-16 22:31:06.139922: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1447] function cupti_interface_->ActivityRegisterCallbacks( AllocCuptiActivityBuffer, FreeCuptiActivityBuffer)failed with error CUPTI could not be loaded or symbol could not be found.
2020-09-16 22:31:06.139933: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1430] function cupti_interface_->EnableCallback( 0 , subscriber_, CUPTI_CB_DOMAIN_DRIVER_API, cbid)failed with error CUPTI


Epoch 00049: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-06.
Epoch 50/100
1195/1195 [==============================] - 678s 567ms/step - loss: 1.2057 - regression_loss: 0.9739 - classification_loss: 0.2317 - val_loss: 1.5665 - val_regression_loss: 1.0022 - val_classification_loss: 0.9953
Running network: 100% (612 of 612) |#####| Elapsed Time: 0:00:51 Time:  0:00:51
Parsing annotations: 100% (612 of 612) |#| Elapsed Time: 0:00:00 Time:  0:00:00
218 instances of class Vehículo with average precision: 0.9477 TP: 213 FP: 522 precision: 0.2898 recall: 0.9771 f1_score: 0.4470 f2_score: 0.6627
16 instances of class Camión with average precision: 0.4977 TP: 15 FP: 960 precision: 0.0154 recall: 0.9375 f1_score: 0.0303 f2_score: 0.0722
7 instances of class Tractor with average precision: 0.2309 TP: 6 FP: 515 precision: 0.0115 recall: 0.8571 f1_score: 0.0227 f2_score: 0.0546
8 instances of class Maquinaria with average precision: 0.8349 TP: 8 FP: 91 precision: 0.0808 recall: 

Epoch 55/100
1195/1195 [==============================] - 658s 550ms/step - loss: 1.0921 - regression_loss: 0.9175 - classification_loss: 0.1745 - val_loss: 1.6001 - val_regression_loss: 0.9951 - val_classification_loss: 0.9684
Running network: 100% (612 of 612) |#####| Elapsed Time: 0:00:51 Time:  0:00:51
Parsing annotations: 100% (612 of 612) |#| Elapsed Time: 0:00:00 Time:  0:00:00
218 instances of class Vehículo with average precision: 0.9586 TP: 215 FP: 533 precision: 0.2874 recall: 0.9862 f1_score: 0.4451 f2_score: 0.6636
16 instances of class Camión with average precision: 0.4501 TP: 14 FP: 758 precision: 0.0181 recall: 0.8750 f1_score: 0.0355 f2_score: 0.0837
7 instances of class Tractor with average precision: 0.2141 TP: 7 FP: 489 precision: 0.0141 recall: 1.0000 f1_score: 0.0278 f2_score: 0.0668
8 instances of class Maquinaria with average precision: 0.7365 TP: 7 FP: 46 precision: 0.1321 recall: 0.8750 f1_score: 0.2295 f2_score: 0.4118
68 instances of class Animal with averag

...


Epoch 00078: ReduceLROnPlateau reducing learning rate to 4.999999719812465e-12.
Epoch 79/100
1195/1195 [==============================] - 643s 538ms/step - loss: 0.9943 - regression_loss: 0.8692 - classification_loss: 0.1251 - val_loss: 1.5890 - val_regression_loss: 0.9958 - val_classification_loss: 0.9943
Running network: 100% (612 of 612) |#####| Elapsed Time: 0:00:52 Time:  0:00:52
Parsing annotations: 100% (612 of 612) |#| Elapsed Time: 0:00:00 Time:  0:00:00
218 instances of class Vehículo with average precision: 0.9590 TP: 215 FP: 493 precision: 0.3037 recall: 0.9862 f1_score: 0.4644 f2_score: 0.6804
16 instances of class Camión with average precision: 0.5510 TP: 14 FP: 770 precision: 0.0179 recall: 0.8750 f1_score: 0.0350 f2_score: 0.0825
7 instances of class Tractor with average precision: 0.4146 TP: 7 FP: 486 precision: 0.0142 recall: 1.0000 f1_score: 0.0280 f2_score: 0.0672
8 instances of class Maquinaria with average precision: 0.7287 TP: 7 FP: 33 precision: 0.1750 recall: 

Parsing annotations: 100% (612 of 612) |#| Elapsed Time: 0:00:00 Time:  0:00:00
218 instances of class Vehículo with average precision: 0.9590 TP: 215 FP: 493 precision: 0.3037 recall: 0.9862 f1_score: 0.4644 f2_score: 0.6804
16 instances of class Camión with average precision: 0.5510 TP: 14 FP: 770 precision: 0.0179 recall: 0.8750 f1_score: 0.0350 f2_score: 0.0825
7 instances of class Tractor with average precision: 0.4146 TP: 7 FP: 486 precision: 0.0142 recall: 1.0000 f1_score: 0.0280 f2_score: 0.0672
8 instances of class Maquinaria with average precision: 0.7287 TP: 7 FP: 33 precision: 0.1750 recall: 0.8750 f1_score: 0.2917 f2_score: 0.4861
68 instances of class Animal with average precision: 0.7511 TP: 60 FP: 1859 precision: 0.0313 recall: 0.8824 f1_score: 0.0604 f2_score: 0.1369
11 instances of class Tuberia with average precision: 0.1378 TP: 7 FP: 3086 precision: 0.0023 recall: 0.6364 f1_score: 0.0045 f2_score: 0.0112
375 instances of class Escombro with average precision: 0.2796


Epoch 00087: ReduceLROnPlateau reducing learning rate to 4.999999488741877e-16.
Epoch 88/100
1195/1195 [==============================] - 622s 520ms/step - loss: 1.0164 - regression_loss: 0.8735 - classification_loss: 0.1429 - val_loss: 1.5890 - val_regression_loss: 0.9958 - val_classification_loss: 0.9943
Running network: 100% (612 of 612) |#####| Elapsed Time: 0:00:51 Time:  0:00:51
Parsing annotations: 100% (612 of 612) |#| Elapsed Time: 0:00:00 Time:  0:00:00
218 instances of class Vehículo with average precision: 0.9590 TP: 215 FP: 493 precision: 0.3037 recall: 0.9862 f1_score: 0.4644 f2_score: 0.6804
16 instances of class Camión with average precision: 0.5510 TP: 14 FP: 770 precision: 0.0179 recall: 0.8750 f1_score: 0.0350 f2_score: 0.0825
7 instances of class Tractor with average precision: 0.4146 TP: 7 FP: 486 precision: 0.0142 recall: 1.0000 f1_score: 0.0280 f2_score: 0.0672
8 instances of class Maquinaria with average precision: 0.7287 TP: 7 FP: 33 precision: 0.1750 recall: 

Running network: 100% (612 of 612) |#####| Elapsed Time: 0:00:51 Time:  0:00:51
Parsing annotations: 100% (612 of 612) |#| Elapsed Time: 0:00:00 Time:  0:00:00
218 instances of class Vehículo with average precision: 0.9590 TP: 215 FP: 493 precision: 0.3037 recall: 0.9862 f1_score: 0.4644 f2_score: 0.6804
16 instances of class Camión with average precision: 0.5510 TP: 14 FP: 770 precision: 0.0179 recall: 0.8750 f1_score: 0.0350 f2_score: 0.0825
7 instances of class Tractor with average precision: 0.4146 TP: 7 FP: 486 precision: 0.0142 recall: 1.0000 f1_score: 0.0280 f2_score: 0.0672
8 instances of class Maquinaria with average precision: 0.7287 TP: 7 FP: 33 precision: 0.1750 recall: 0.8750 f1_score: 0.2917 f2_score: 0.4861
68 instances of class Animal with average precision: 0.7511 TP: 60 FP: 1859 precision: 0.0313 recall: 0.8824 f1_score: 0.0604 f2_score: 0.1369
11 instances of class Tuberia with average precision: 0.1378 TP: 7 FP: 3086 precision: 0.0023 recall: 0.6364 f1_score: 0.0045

Epoch 97/100
1195/1195 [==============================] - 636s 532ms/step - loss: 1.0180 - regression_loss: 0.8760 - classification_loss: 0.1420 - val_loss: 1.5890 - val_regression_loss: 0.9958 - val_classification_loss: 0.9943
Running network: 100% (612 of 612) |#####| Elapsed Time: 0:00:51 Time:  0:00:51
Parsing annotations: 100% (612 of 612) |#| Elapsed Time: 0:00:00 Time:  0:00:00
218 instances of class Vehículo with average precision: 0.9590 TP: 215 FP: 493 precision: 0.3037 recall: 0.9862 f1_score: 0.4644 f2_score: 0.6804
16 instances of class Camión with average precision: 0.5510 TP: 14 FP: 770 precision: 0.0179 recall: 0.8750 f1_score: 0.0350 f2_score: 0.0825
7 instances of class Tractor with average precision: 0.4146 TP: 7 FP: 486 precision: 0.0142 recall: 1.0000 f1_score: 0.0280 f2_score: 0.0672
8 instances of class Maquinaria with average precision: 0.7287 TP: 7 FP: 33 precision: 0.1750 recall: 0.8750 f1_score: 0.2917 f2_score: 0.4861
68 instances of class Animal with averag

# DEBUG

If the model is not learning or has low accuracy, check the dataset and the anchors of the model to determine if the problem lies in a mismatch between the dataset and the anchors.

In [47]:
!python ../../keras-retinanet/keras_retinanet/bin/debug.py \
--config {CONFIG_FILE} \
--show-annotations \
csv {TRAIN_ANNOTATIONS} {CLASSES_FILE}

Using TensorFlow backend.
Qt: Session management error: Could not open network socket
^C


In [84]:
%pwd

'/home/aikauel/enap'

# Evaluate model

Evaluate model with test dataset (test.csv)

In [130]:
# First, create inference model
# path to trained weights
epoch = 61
last_model = os.path.join(SNAPSHOTS_DIR, 'resnet50_csv_{}.h5'.format(epoch))
inference_model = last_model.replace('snapshots', 'inference')

# convert trained model to inference model to be used for generating predictions
inference_model = last_model.replace('snapshots/', 'inference/')
!python ../../keras-retinanet/keras_retinanet/bin/convert_model.py {last_model} {inference_model} \
--config {CONFIG_FILE}

Using TensorFlow backend.
2020-09-16 22:21:06.206069: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-09-16 22:21:06.234835: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-09-16 22:21:06.235227: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.755GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2020-09-16 22:21:06.235386: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-09-16 22:21:06.236351: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2020-09-16 22:21:06.237306: I tensorf

In [63]:
#TEST_ANNOTATIONS = os.path.join(DATASET_VERSION_DIR, 'test.csv')
TEST_ANNOTATIONS = os.path.join(DATASET_VERSION_DIR, 'test.csv')

In [131]:
#output_images_path = os.path.join(TRAINING_BASE_DIR, 'test_output')

# evaluate model on test.csv
!python ../../keras-retinanet/keras_retinanet/bin/evaluate.py \
--config {CONFIG_FILE} \
--score-threshold 0.35 \
--iou-threshold 0.35 \
--no-resize \
--max-detections 300 \
csv {TEST_ANNOTATIONS} \
{CLASSES_FILE} \
{inference_model}

Using TensorFlow backend.
Loading model, this may take a second...
2020-09-16 22:21:20.231116: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-09-16 22:21:20.259592: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-09-16 22:21:20.259910: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.755GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2020-09-16 22:21:20.260052: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-09-16 22:21:20.261005: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so

/home/aikauel/anaconda3/envs/enap/lib/python3.6/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
Running network: N/A% (0 of 612) |       | Elapsed Time: 0:00:00 ETA:  --:--:--2020-09-16 22:21:23.278742: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2020-09-16 22:21:24.023666: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
Running network: 100% (612 of 612) |#####| Elapsed Time: 0:00:53 Time:  0:00:53
Parsing annotations: 100% (612 of 612) |#| Elapsed Time: 0:00:00 Time:  0:00:00
218 instances of class 0 with average precision: 0.9307 TP: 206 FP: 56 precision: 0.7863 recall: 0.9450 f1_score: 0.8583 f2_score: 0.9083
16 instances of class 1 with average precision: 0.4375 TP: 7 FP: 0 preci

## Test model
#### Test model on images without annotations (unseen in train-val-test)

#### Convert training model to inference model

In [6]:
os.getcwd()

'/home/aikauel/enap/aerialnet_project/notebooks'

In [126]:
SNAPSHOTS_DIR

'../trainings/v4/snapshots2'

In [138]:
# First, create inference model
# path to trained weights
epoch = 61
last_model = os.path.join(SNAPSHOTS_DIR, 'resnet50_csv_{}.h5'.format(epoch))
inference_model = last_model.replace('snapshots', 'inference')

# convert trained model to inference model to be used for generating predictions
inference_model = last_model.replace('snapshots/', 'inference/')
!python ../../keras-retinanet/keras_retinanet/bin/convert_model.py {last_model} {inference_model} \
--config {CONFIG_FILE}

Using TensorFlow backend.
2020-09-17 10:50:37.374362: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-09-17 10:50:37.417822: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-09-17 10:50:37.418347: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.755GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2020-09-17 10:50:37.418595: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-09-17 10:50:37.420229: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2020-09-17 10:50:37.421820: I tensorf

#### Set paths

In [139]:
test_images_path = '/home/aikauel/enap/aerialnet_project/datasets/official_37_test_images/'
#test_images_path = '/home/aikauel/enap/data/vitto/'
test_output_path = os.path.join(TRAINING_BASE_DIR, 
                                'official_37_output_epoch{}_threshold53'.format(epoch))

In [140]:
test_output_path

'../trainings/v4/official_37_output_epoch61_threshold53'

In [141]:
!python ../../aerialnet_project/shared_utils/image_inference_write.py \
--input_dir {test_images_path} \
--model {inference_model} \
--output_dir {test_output_path} \
--labels {CLASSES_FILE} \
--threshold 0.53 \
--no_csv True

Using TensorFlow backend.
/home/aikauel/enap/aerialnet_project/datasets/official_37_test_images/
LABELS: {0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9'}
2020-09-17 10:51:00.254701: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-09-17 10:51:00.283972: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-09-17 10:51:00.284302: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.755GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2020-09-17 10:51:00.284470: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-09-17 10:51:00.2

/home/aikauel/anaconda3/envs/enap/lib/python3.6/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
no csv: /home/aikauel/enap/aerialnet_project/datasets/official_37_test_images/
[INFO] predicting image 1 of 37: 2020-08-11_08:13:26-3872762-DJI_0282.jpg
2020-09-17 10:51:03.399938: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2020-09-17 10:51:04.507446: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
processing time:  5.141803503036499
../trainings/v4/official_37_output_epoch61_threshold53/2020-08-11_08:13:26-3872762-DJI_0282.jpg
[INFO] predicting image 2 of 37: 2020-08-11_08:15:40-3618600-DJI_0296.jpg
processing time:  0.4231247901916504
../trainings/v4/official_37_output_epoch61_threshold5

# Create saved_model to load tfserving model

In [42]:
last_model = os.path.join(SNAPSHOTS_DIR, 'resnet50_csv_107.h5')

saved_model = os.path.dirname(last_model).replace('snapshots_iter2', 'saved_models')

In [43]:
print(last_model, saved_model)

../trainings/v3/snapshots_iter2/resnet50_csv_107.h5 ../trainings/v3/saved_models


In [44]:
!python ../shared_utils/convert_model.py {last_model} {saved_model} \
--config {CONFIG_FILE}

Using TensorFlow backend.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
2020-09-01 18:34:37.042366: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-09-01 18:34:37.074438: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-09-01 18:34:37.074740: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.755GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2020-09-01 18:34:37.074883: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-09-01 18:34:37.075773: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successf

[INFO] Model saved


## Visualization
#### Generate detections on test images - IMAGES

In [46]:
os.getcwd()

'/home/aikauel/enap'

In [13]:
# create output directory where you want to save images with bounding boxes
#!mkdir /content/data/output

# generate detections on images
!python ije_retinanet/image_inference_print.py \
-i 'data/test_images' \
-t 0.6 \
-m  {inference_model} \
-o data/output

python: can't open file 'ije_retinanet/image_inference_print.py': [Errno 2] No such file or directory
